<a href="https://colab.research.google.com/github/MDfarazuddin99/CNN_Pruning/blob/master/VGG_HISTORY_L1_PRUNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%tensorflow_version 1.x
# !pip install tesnsorflow 1.x
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

import keras
from keras.datasets import cifar10
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, GlobalAveragePooling2D,BatchNormalization,Activation
from keras.models import load_model
from keras.callbacks import Callback
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.layers.core import Lambda
from keras import backend as K
from keras import regularizers
from keras.callbacks import ModelCheckpoint



!pip install kerassurgeon
from kerassurgeon import identify 
from kerassurgeon.operations import delete_channels,delete_layer
from kerassurgeon import Surgeon


In [0]:
 def my_get_all_conv_layers(model , first_time):

    '''
    Arguments:
        model -> your model
        first_time -> type boolean 
            first_time = True => model is not pruned 
            first_time = False => model is pruned
    Return:
        List of Indices containing convolution layers
    '''

    all_conv_layers = list()
    for i,each_layer in enumerate(model.layers):
        if (each_layer.name[0:6] == 'conv2d'):
            all_conv_layers.append(i)
    return all_conv_layers if (first_time==True) else all_conv_layers[1:]


def my_get_all_dense_layers(model):
    '''
    Arguments:
        model -> your model        
    Return:
        List of Indices containing fully connected layers
    '''
    all_dense_layers = list()
    for i,each_layer in enumerate(model.layers):
        if (each_layer.name[0:5] == 'dense'):
            all_dense_layers.append(i)
    return all_dense_layers




def count_conv_params_flops(conv_layer):
    # out shape is  n_cells_dim1 * (n_cells_dim2 * n_cells_dim3)
    '''
    Arguments:
        conv layer 
    Return:
        Number of Parameters, Number of Flops
    '''
    
    
    out_shape = conv_layer.output.shape.as_list()
    n_cells_total = np.prod(out_shape[1:-1])

    n_conv_params_total = conv_layer.count_params()

    conv_flops = n_conv_params_total * n_cells_total

 

    return n_conv_params_total, conv_flops


def count_dense_params_flops(dense_layer):
    # out shape is  n_cells_dim1 * (n_cells_dim2 * n_cells_dim3)
    '''
    Arguments:
      dense layer 
    Return:
        Number of Parameters, Number of Flops

    '''

    out_shape = dense_layer.output.shape.as_list()
    n_cells_total = np.prod(out_shape[1:-1])

    n_dense_params_total = dense_layer.count_params()

    dense_flops = n_dense_params_total


    return n_dense_params_total, dense_flops




def count_model_params_flops(model,first_time):

    '''
    Arguments:
        model -> your model
        first_time -> boolean variable
        first_time = True => model is not pruned 
        first_time = False => model is pruned
    Return:
        Number of parmaters, Number of Flops
    '''

    total_params = 0
    total_flops = 0
    # if first_time == True:
    #     model_layers = model.layers
    # else:
    #     model_layers = model.layers[1:-2]
    # all_conv_layers = my_get_all_conv_layers(model,first_time)
    # all_dense_layers = my_get_all_dense_layers(model)
    # model_layers = list()
    
    # for i in all_conv_layers:
    #     model_layers.append(model.layers[i])
    # for i in all_dense_layers:
    #     model_layers.append(model.layers[i])
    model_layers = model.layers[1:-2]
    for index,layer in enumerate(model_layers):
        if any(conv_type in str(type(layer)) for conv_type in ['Conv1D', 'Conv2D', 'Conv3D']):
            print(index,layer.name)
            params, flops = count_conv_params_flops(layer)
            total_params += params
            total_flops += flops
        elif 'Dense' in str(type(layer)):
            print(index,layer.name) 
            params, flops = count_dense_params_flops(layer)
            total_params += params
            total_flops += flops
    return total_params, total_flops

def my_get_weights_in_conv_layers(model,first_time):

    '''
    Arguments:
        model -> your model
        first_time -> boolean variable
            first_time = True => model is not pruned 
            first_time = False => model is pruned
    Return:
        List containing weight tensors of each layer
    '''
    


  weights = list()
  all_conv_layers = my_get_all_conv_layers(model,first_time)
  layer_wise_weights = list() 
  for i in all_conv_layers:
        weights.append(model.layers[i].get_weights()[0])  
  return weights

def my_get_l1_norms_filters_per_epoch(weight_list_per_epoch):

    '''
    Arguments:
        List
    Return:
        Number of parmaters, Number of Flops
    '''
    
    # weight_list_per_epoch = my_get_weights_in_conv_layers(model,first_time)
    l1_norms_filters_per_epoch = list()
    

    for index in range(len(weight_list_per_epoch)):

        epochs = np.array(weight_list_per_epoch[index]).shape[0]
        h , w , d = np.array(weight_list_per_epoch[index]).shape[1], np.array(weight_list_per_epoch[index]).shape[2] , np.array(weight_list_per_epoch[index]).shape[3]


        l1_norms_filters_per_epoch.append(np.sum(np.array(weight_list_per_epoch[index]).reshape(epochs,h*w*d,-1),axis=1))
    return l1_norms_filters_per_epoch

def my_in_conv_layers_get_sum_of_l1_norms_sorted_indices(weight_list_per_epoch):
    layer_wise_filter_sorted_indices = list()
    layer_wise_filter_sorted_values = list()
    l1_norms_filters_per_epoch = my_get_l1_norms_filters_per_epoch(weight_list_per_epoch)
    sum_l1_norms = list()
    
    for i in l1_norms_filters_per_epoch:
        sum_l1_norms.append(np.sum(i,axis=0))
    
    layer_wise_filter_sorted_indices = list()
    
    for i in sum_l1_norms:
        a = pd.Series(i).sort_values().index
        layer_wise_filter_sorted_indices.append(a.tolist())
    return layer_wise_filter_sorted_indices


def my_get_percent_prune_filter_indices(layer_wise_filter_sorted_indices,percentage):    

    prune_filter_indices = list()
    for i in range(len(layer_wise_filter_sorted_indices)):
        prune_filter_indices.append(int(len(layer_wise_filter_sorted_indices[i]) * (percentage/100))   )
    return prune_filter_indices



def my_delete_filters(model,weight_list_per_epoch,percentage,first_time):

    sum_of_l1_norms_sorted_indices = my_in_conv_layers_get_sum_of_l1_norms_sorted_indices(weight_list_per_epoch)

    layer_wise_filter_sorted_indices = my_in_conv_layers_get_sum_of_l1_norms_sorted_indices(weight_list_per_epoch)
    # print(layer_wise_filter_sorted_indices)
    prune_filter_indices = my_get_percent_prune_filter_indices(layer_wise_filter_sorted_indices,percentage)
    # print(prune_filter_indices)
    all_conv_layers = my_get_all_conv_layers(model,first_time)

    surgeon = Surgeon(model)
    for index,value in enumerate(all_conv_layers):
        # print(index,value,layer_wise_filter_sorted_indices[index][0:prune_filter_indices[index]])
        surgeon.add_job('delete_channels',model.layers[value],channels = layer_wise_filter_sorted_indices[index][0:prune_filter_indices[index]])

    model_new = surgeon.operate()
 
    
    return model_new



In [0]:

class Get_Weights(Callback):
    def __init__(self,first_time):
        super(Get_Weights, self).__init__()
        self.weight_list = [] #Using a list of list to store weight tensors per epoch
        self.first_time = first_time
    def on_epoch_end(self,epoch,logs=None):
        if epoch == 0:
            all_conv_layers = my_get_all_conv_layers(self.model,self.first_time)
            for i in range(len(all_conv_layers)):
                self.weight_list.append([]) # appending empty lists for later appending weight tensors 
        
        for index,each_weight in enumerate(my_get_weights_in_conv_layers(self.model,self.first_time)):
                self.weight_list[index].append(each_weight)  



class cifar10vgg:

    def __init__(self,first_time,epochs,train=True):
        self.epochs = epochs
        self.first_time = first_time
        self.num_classes = 10
        self.weight_decay = 0.0005
        self.x_shape = [32,32,3]
        self.history = 0
        self.weight_list_per_epoch = None
        self.model = self.build_model()
        if train:
            self.model, self.history ,self.weight_list_per_epoch = self.train(self.model)
        else:
            self.model.load_weights('drive/My Drive/Colab Notebooks/cifar10vgg.h5')


    def build_model(self):
        # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

        model = Sequential()
        weight_decay = self.weight_decay

        model.add(Conv2D(64, (3, 3), padding='same',
                         input_shape=self.x_shape,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))


        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

        model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.5))

        model.add(Flatten())
        model.add(Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Dropout(0.5))
        model.add(Dense(self.num_classes))
        model.add(Activation('softmax'))
        return model


    def normalize(self,X_train,X_test):
        #this function normalize inputs for zero mean and unit variance
        # it is used when training a model.
        # Input: training set and test set
        # Output: normalized training set and test set according to the trianing set statistics.
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test


    def train(self,model):

        #training parameters
        batch_size = 128
        maxepoches = 250
        learning_rate = 0.1
        lr_decay = 1e-6
        lr_drop = 20
        # The data, shuffled and split between train and test sets:
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)

        def lr_scheduler(epoch):
            return learning_rate * (0.5 ** (epoch // lr_drop))
        reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

        save_model = ModelCheckpoint('drive/My Drive/Colab Notebooks/pruned_cifarvgg.h5', monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

        #data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images
        # (std, mean, and principal components if ZCA whitening is applied).
        # datagen.fit(x_train)



        #optimization details
        sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
        model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

        gw = Get_Weights(self.first_time)

        # training process in a for loop with learning rate drop every 25 epoches.

        history = model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=self.epochs,
                            validation_data=(x_test, y_test),callbacks=[reduce_lr, save_model,gw],verbose=1)
        #model.save_weights('cifar10vgg.h5')
        return model, history,gw.weight_list

        

In [0]:
def normalize(X_train,X_test):
    #this function normalize inputs for zero mean and unit variance
    # it is used when training a model.
    # Input: training set and test set
    # Output: normalized training set and test set according to the trianing set statistics.
    mean = np.mean(X_train,axis=(0,1,2,3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train-mean)/(std+1e-7)
    X_test = (X_test-mean)/(std+1e-7)
    return X_train, X_test


def train(model,epochs):

    #training parameters
    batch_size = 128
    maxepoches = 250
    learning_rate = 0.1
    lr_decay = 1e-6
    lr_drop = 20

    num_classes = 10
    weight_decay = 0.0005
    x_shape = [32,32,3]

    # The data, shuffled and split between train and test sets:
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train, x_test = normalize(x_train, x_test)

    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))
    reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

    save_model = ModelCheckpoint('drive/My Drive/Colab Notebooks/pruned_cifarvgg.h5', monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

    #data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # (std, mean, and principal components if ZCA whitening is applied).
    # datagen.fit(x_train)



    #optimization details
    sgd = optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])

    gw = Get_Weights(False)

    # training process in a for loop with learning rate drop every 25 epoches.

    history = model.fit_generator(datagen.flow(x_train, y_train,
                                        batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),callbacks=[reduce_lr, save_model,gw],verbose=1)
    #model.save_weights('cifar10vgg.h5')
    return model, history,gw.weight_list


In [15]:
# m = my_delete_filters(my_model,weight_list_per_epoch,30,True)
# train for first time
my_vgg = cifar10vgg(first_time=True,epochs=20)
model, history ,weight_list_per_epoch= my_vgg.model, my_vgg.history, my_vgg.weight_list_per_epoch


#this dictionary is to log the parameters and is later converted into a dataframe.
log_dict = dict()
log_dict['train_loss'] = []
log_dict['train_acc'] = []
log_dict['val_loss'] = []
log_dict['val_acc'] = []
log_dict['total_params'] = []
log_dict['total_flops'] = []

best_acc_index = history.history['val_acc'].index(max(history.history['val_acc']))
log_dict['train_loss'].append(history.history['loss'][best_acc_index])
log_dict['train_acc'].append(history.history['acc'][best_acc_index])
log_dict['val_loss'].append(history.history['val_loss'][best_acc_index])
log_dict['val_acc'].append(history.history['val_acc'][best_acc_index])
a,b = count_model_params_flops(model,True)
log_dict['total_params'].append(a)
log_dict['total_flops'].append(b)


Epoch 1/20
390/390 [==============================] - 57s 145ms/step - loss: 19.5526 - acc: 0.2146 - val_loss: 19.6632 - val_acc: 0.1281
Epoch 2/20
390/390 [==============================] - 50s 127ms/step - loss: 10.5674 - acc: 0.2966 - val_loss: 8.8999 - val_acc: 0.1608
Epoch 3/20
390/390 [==============================] - 50s 127ms/step - loss: 7.0928 - acc: 0.2637 - val_loss: 6.4324 - val_acc: 0.2408
Epoch 4/20
390/390 [==============================] - 50s 128ms/step - loss: 7.3277 - acc: 0.2789 - val_loss: 7.3206 - val_acc: 0.1694
Epoch 5/20
390/390 [==============================] - 50s 127ms/step - loss: 5.5498 - acc: 0.3499 - val_loss: 4.7100 - val_acc: 0.3092
Epoch 6/20
390/390 [==============================] - 50s 127ms/step - loss: 3.5751 - acc: 0.4543 - val_loss: 3.3164 - val_acc: 0.3322
Epoch 7/20
390/390 [==============================] - 50s 127ms/step - loss: 2.4349 - acc: 0.5341 - val_loss: 2.0955 - val_acc: 0.5744
Epoch 8/20
390/390 [==============================] 

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_16 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_11 (Dropout)         (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_17 (Activation)   (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 32, 32, 64)       

In [17]:

#stop pruning if the accuracy drops by 5% from maximum accuracy ever obtained. 
validation_accuracy = max(history.history['val_acc'])
max_val_acc = validation_accuracy
count = 0
all_models = list()
while validation_accuracy - max_val_acc >= -0.05 :

# # while max_val_acc <= validation_accuracy  :

    print("ITERATION {} ".format(count+1))
    all_models.append(model)
    if max_val_acc < validation_accuracy:
        max_val_acc = validation_accuracy
        

    if count < 1:
        model = my_delete_filters(model,weight_list_per_epoch,10,True)

   
    else:
        model = my_delete_filters(model,weight_list_per_epoch,10,False)
    
    
    a,b = count_model_params_flops(model,False)
    model,history,weight_list_per_epoch = train(model,10)

    validation_accuracy = max(history.history['val_acc'])
    best_acc_index = history.history['val_acc'].index(max(history.history['val_acc']))
    log_dict['train_loss'].append(history.history['loss'][best_acc_index])
    log_dict['train_acc'].append(history.history['acc'][best_acc_index])
    log_dict['val_loss'].append(history.history['val_loss'][best_acc_index])
    log_dict['val_acc'].append(history.history['val_acc'][best_acc_index])
    log_dict['total_params'].append(a)
    log_dict['total_flops'].append(b)

    print("VALIDATION ACCURACY AFTER {} ITERATIONS = {}".format(count+1,validation_accuracy))
    count+=1


ITERATION 1 
Deleting 6/64 channels from layer: conv2d_14
Deleting 6/64 channels from layer: conv2d_15
Deleting 12/128 channels from layer: conv2d_16
Deleting 12/128 channels from layer: conv2d_17
Deleting 25/256 channels from layer: conv2d_18
Deleting 25/256 channels from layer: conv2d_19
Deleting 25/256 channels from layer: conv2d_20
Deleting 51/512 channels from layer: conv2d_21
Deleting 51/512 channels from layer: conv2d_22
Deleting 51/512 channels from layer: conv2d_23
Deleting 51/512 channels from layer: conv2d_24
Deleting 51/512 channels from layer: conv2d_25
Deleting 51/512 channels from layer: conv2d_26
0 conv2d_14
4 conv2d_15
8 conv2d_16
12 conv2d_17
16 conv2d_18
20 conv2d_19
24 conv2d_20
28 conv2d_21
32 conv2d_22
36 conv2d_23
40 conv2d_24
44 conv2d_25
48 conv2d_26
54 dense_3
Epoch 1/10
390/390 [==============================] - 53s 136ms/step - loss: 1.4626 - acc: 0.7226 - val_loss: 1.5553 - val_acc: 0.7239
Epoch 2/10
390/390 [==============================] - 46s 117ms/step

In [0]:
log_df = pd.DataFrame(log_dict)
log_df.to_csv('/content/drive/My Drive/RESULT_VGG_HISTORY_L1_PRUNING.csv')